In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#### NOTE: This code was run on colab and some of the sweeps were interupted, so I ended rerunning some sweeps for a total of 5 which is why some outputs are shorter or error prone.

# Data Loading Both Classes

In [ ]:
import tensorflow as tf


In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)


In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow-gpu'


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def load_data(data_dir, target_size=(224, 224)):
    X = []
    y = []

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                image = Image.open(file_path).convert('RGB')
                resized_image = image.resize(target_size)
                X.append(np.array(resized_image))
                y.append(folder_name)

    return np.array(X), np.array(y)

# Step 1: Download the dataset from Kaggle and unzip the file

# Step 2: Load the images and labels
data_dir = '/content/drive/Othercomputers/My Laptop/lakehouse/Fruits_Vegetables_Dataset(12000)'
X, y = load_data(data_dir)

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 4: Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

# Step 5: Define image transformations for standardization and data augmentation
image_size = X_train.shape[1:]

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow(X_train, y_train_encoded, batch_size=64)
test_generator = test_datagen.flow(X_test, y_test_encoded, batch_size=64)


In [ ]:
# Get a batch from the train_generator
X_batch, y_batch = next(train_generator)

# Print some values from the training batch
print("Training batch data:")
print("Batch images shape:", X_batch.shape)
print("Batch labels shape:", y_batch.shape)
print("First image of the batch:", X_batch[30])
print("Label of the first image:", y_batch[30])

# Get a batch from the test_generator
X_batch_test, y_batch_test = next(test_generator)

# Print some values from the testing batch
print("\nTesting batch data:")
print("Batch images shape:", X_batch_test.shape)
print("Batch labels shape:", y_batch_test.shape)
print("First image of the batch:", X_batch_test[30])
print("Label of the first image:", y_batch_test[30])


Training batch data:
Batch images shape: (64, 224, 224, 3)
Batch labels shape: (64,)
First image of the batch: [[[0.45956445 0.41409597 0.33100212]
  [0.46740124 0.41801435 0.3317398 ]
  [0.475238   0.41960788 0.33565822]
  ...
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]]

 [[0.45988008 0.41425377 0.3308443 ]
  [0.4677169  0.41817215 0.33189765]
  [0.47555366 0.41960788 0.33581606]
  ...
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]]

 [[0.46019575 0.4144116  0.33068648]
  [0.4680325  0.41832998 0.33205548]
  [0.4758693  0.41960788 0.33597386]
  ...
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]
  [0.82745105 0.8862746  0.9686275 ]]

 ...

 [[1.         1.         0.9921569 ]
  [1.         1.         0.9921569 ]
  [1.         1.         0.9921569 ]
  ...
  [0.13355389 0.10284298 0.10218134]
  [0.13339607 0.1022117  0.10202352]
  

In [ ]:
unique_labels, counts = np.unique(y_train_encoded, return_counts=True)

num_unique_labels = len(unique_labels)

print(f"The number of unique labels in 'y': {num_unique_labels}")
print(f"Unique labels: {unique_labels}")
print(f"Counts for each label: {counts}")


The number of unique labels in 'y': 20
Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Counts for each label: [490 499 489 496 486 484 487 492 483 483 470 461 473 464 474 474 473 468
 477 477]


# Classifying both at the same time

### Lenet-5

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ad294afc068828b9785b43b2175c0e5e6a9771484d6f6dea5ae08795b03312b5
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

sweep_config = {
    'method': 'random',
    'parameters': {
        'learning_rate': {'values': [0.0001, 0.00025, 0.0005, 0.001]},
        'batch_size': {'values': [32,64,128]},
        'dropout': {'values': [0.0, 0.1, 0.2, 0.5]},
    }
}


In [ ]:
import pprint

metric = {
    'name': 'val_accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'batch_size': {'values': [32, 64, 128]},
                'dropout': {'values': [0.0, 0.1, 0.2, 0.5]},
                'learning_rate': {'values': [0.0001, 0.00025, 0.0005, 0.001]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project='LenetBothtest')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
import statistics


# Step 1: Define the LeNet-5 architecture
def WBcreate_lenet5_model(input_shape, num_classes, learning_rate, dropout):
    model = Sequential()
    model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(84, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Step 2: train

def WBtrain(config=None):
    input_shape = (224, 224, 3)
    num_classes = len(np.unique(y))

    with wandb.init(config=config):
        config = wandb.config

        lr = config.learning_rate
        batch_size = config.batch_size
        dropout = config.dropout

        lenet5_model = WBcreate_lenet5_model(input_shape, num_classes, learning_rate=lr, dropout=dropout)

        # Step 4: Train the model
        history = lenet5_model.fit(train_generator,
                                  epochs=10,
                                  batch_size=batch_size,
                                  validation_data=test_generator,
                                  callbacks=[WandbCallback()]) # add wandb callback for logging

        # Save loss value
        wandb.log({"val_loss": statistics.mean(history.history['val_loss'])})
        wandb.log({"val_accuracy": statistics.mean(history.history['val_accuracy'])})

In [ ]:
wandb.agent(sweep_id, WBtrain, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


 84/150 [===============>..............] - ETA: 41s - loss: 1.3945 - accuracy: 0.5549

wandb: Agent Starting Run: z1mwojvm with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.0005


 95/150 [==================>...........] - ETA: 35s - loss: 1.3876 - accuracy: 0.5594

 55/150 [==========>...................] - ETA: 1:12 - loss: 2.9540 - accuracy: 0.0858

...

wandb: Adding directory to artifact (/content/wandb/run-20230510_123508-ukcb9hlg/files/model-best)... Done. 0.0s


150/150 [==============================] - 95s 631ms/step - loss: 2.0752 - accuracy: 0.3583 - val_loss: 1.7837 - val_accuracy: 0.4625


accuracy,▁▃▄▅▆▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▆▅▄▃▂▂▁▁
val_accuracy,▁▂▄▄▅▆▆▇██▅
val_loss,█▇▅▄▄▃▂▂▁▁▄
accuracy,0.35833
best_epoch,9
best_val_loss,1.78373
epoch,9
loss,2.07517
val_accuracy,0.33033


Error in callback <function _WandbInit._pause_backend at 0x7fb5a8520b80> (for post_run_cell):


BrokenPipeError: ignored

### VGG 16

In [ ]:
import pprint
sweep_id = wandb.sweep(sweep_config, project='VGG16Bothtest')
pprint.pprint(sweep_config)

Create sweep with ID: byvlx6ht
Sweep URL: https://wandb.ai/jdc690nyu/VGG16Bothtest/sweeps/byvlx6ht
{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'batch_size': {'values': [32, 64, 128]},
                'dropout': {'values': [0.0, 0.1, 0.2, 0.5]},
                'learning_rate': {'values': [0.0001, 0.00025, 0.0005, 0.001]}}}


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import statistics
# Step 1: Load the pre-trained VGG-16 model without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def WBcreate_VGGmodel(num_classes, learning_rate, dropout):
    # Step 2: Add custom fully connected layers to match the number of classes in your dataset
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout)(x)  # Add a dropout layer with a rate of 0.5
    predictions = Dense(num_classes, activation='softmax')(x)

    # Step 3: Create the final model
    vgg16_model = Model(inputs=base_model.input, outputs=predictions)

    # Step 4: Freeze the base model layers, so only the custom layers will be trained
    for layer in base_model.layers:
        layer.trainable = False

    # Step 5: Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    vgg16_model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return vgg16_model



# Step 8: Define the training function with the WandB callback
def WBtrain(config=None):
    num_classes = len(np.unique(y))

    with wandb.init(config=config):
        config = wandb.config

        lr = config.learning_rate
        batch_size = config.batch_size
        dropout = config.dropout

        vgg16_model = WBcreate_VGGmodel(num_classes, learning_rate=lr, dropout=dropout)

        # Step 4: Train the model
        history = vgg16_model.fit(train_generator,
                                  epochs=10,
                                  batch_size=batch_size,
                                  validation_data=test_generator,
                                  callbacks=[WandbCallback()]) # add wandb callback for logging

        # Save loss value
        wandb.log({"val_loss": statistics.mean(history.history['val_loss'])})
        wandb.log({"val_accuracy": statistics.mean(history.history['val_accuracy'])})


In [ ]:
wandb.agent(sweep_id, WBtrain, count=5)

wandb: Agent Starting Run: k40kp1a9 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 2.8417 - accuracy: 0.1401

wandb: Adding directory to artifact (/content/wandb/run-20230510_131622-k40kp1a9/files/model-best)... Done. 0.3s


150/150 [==============================] - 115s 686ms/step - loss: 2.8417 - accuracy: 0.1401 - val_loss: 2.4847 - val_accuracy: 0.4825
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 2.3698 - accuracy: 0.3489

...

150/150 [==============================] - 102s 680ms/step - loss: 0.3861 - accuracy: 0.8829 - val_loss: 0.3961 - val_accuracy: 0.8804
Epoch 10/10
150/150 [==============================] - ETA: 0s - loss: 0.3522 - accuracy: 0.8949

wandb: Adding directory to artifact (/content/wandb/run-20230510_142521-6mryiizu/files/model-best)... Done. 0.2s


150/150 [==============================] - 103s 684ms/step - loss: 0.3522 - accuracy: 0.8949 - val_loss: 0.3769 - val_accuracy: 0.8829


accuracy,▁▅▆▆▇▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▂▁▁▁
val_accuracy,▁▄▅▆▇▇████
val_loss,█▅▄▃▂▂▂▁▁▁
accuracy,0.89485
best_epoch,9
best_val_loss,0.37693
epoch,9
loss,0.35221
val_accuracy,0.88292


Run 6mryiizu errored: NameError("name 'statistics' is not defined")
wandb: ERROR Run 6mryiizu errored: NameError("name 'statistics' is not defined")


### AlexNet

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='AlexNetBothtest')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 10m5hijw
Sweep URL: https://wandb.ai/jdc690nyu/AlexNetBothtest/sweeps/10m5hijw


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbCallback
import statistics

# Step 1: Define the AlexNet architecture
def WBcreate_alexnet_model(input_shape, num_classes, learning_rate, dropout):
    model = tf.keras.models.Sequential()

    # First Convolutional Layer
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())

    # Second Convolutional Layer
    model.add(Conv2D(256, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())

    # Third Convolutional Layer
    model.add(Conv2D(384, (3, 3), activation='relu'))

    # Fourth Convolutional Layer
    model.add(Conv2D(384, (3, 3), activation='relu'))

    # Fifth Convolutional Layer
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())

    # Flatten the feature maps
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(dropout))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Step 2: Train the model
def WBtrain(config=None):
    input_shape = (224, 224, 3)
    num_classes = len(np.unique(y))

    with wandb.init(config=config):
        config = wandb.config
        lr = config.learning_rate
        batch_size = config.batch_size
        dropout = config.dropout

        alexnet_model = WBcreate_alexnet_model(input_shape, num_classes, learning_rate=lr, dropout=dropout)

        history = alexnet_model.fit(train_generator,
                                    epochs=10,
                                    validation_data=test_generator,
                                    batch_size=batch_size,
                                    callbacks=[WandbCallback()])
        
        # Save loss value
        wandb.log({"val_loss": statistics.mean(history.history['val_loss'])})
        wandb.log({"val_accuracy": statistics.mean(history.history['val_accuracy'])})        


In [ ]:
wandb.agent(sweep_id, WBtrain, count=5)

wandb: Agent Starting Run: qv5u8u85 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: jdc690 (jdc690nyu). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 2.0520 - accuracy: 0.3639

wandb: Adding directory to artifact (/content/wandb/run-20230510_135554-qv5u8u85/files/model-best)... Done. 1.4s


150/150 [==============================] - 126s 730ms/step - loss: 2.0520 - accuracy: 0.3639 - val_loss: 3.5334 - val_accuracy: 0.0796
Epoch 2/10
150/150 [==============================] - 104s 694ms/step - loss: 1.2959 - accuracy: 0.5806 - val_loss: 3.8031 - val_accuracy: 0.1058
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: 1.0402 - accuracy: 0.6619

...

wandb: Adding directory to artifact (/content/wandb/run-20230510_151007-uca14trh/files/model-best)... Done. 1.5s


150/150 [==============================] - 109s 729ms/step - loss: 0.6809 - accuracy: 0.7711 - val_loss: 0.7103 - val_accuracy: 0.7613
Epoch 7/10
150/150 [==============================] - 104s 695ms/step - loss: 0.6091 - accuracy: 0.8008 - val_loss: 0.8994 - val_accuracy: 0.7138
Epoch 8/10
150/150 [==============================] - 103s 684ms/step - loss: 0.5689 - accuracy: 0.8121 - val_loss: 0.7347 - val_accuracy: 0.7671
Epoch 9/10
150/150 [==============================] - 104s 690ms/step - loss: 0.5248 - accuracy: 0.8245 - val_loss: 0.7579 - val_accuracy: 0.7575
Epoch 10/10
150/150 [==============================] - ETA: 0s - loss: 0.5147 - accuracy: 0.8250

wandb: Adding directory to artifact (/content/wandb/run-20230510_151007-uca14trh/files/model-best)... Done. 1.4s


150/150 [==============================] - 109s 726ms/step - loss: 0.5147 - accuracy: 0.8250 - val_loss: 0.5977 - val_accuracy: 0.8037


accuracy,▁▄▆▆▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▃▂▂▁▁▁▁
val_accuracy,▁▃▅▇▆█▇███▆
val_loss,█▅▃▂▂▁▂▁▁▁▃
accuracy,0.825
best_epoch,9
best_val_loss,0.59774
epoch,9
loss,0.51471
val_accuracy,0.58638


## InceptionResNet2

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='Iv2Bothtest')

Create sweep with ID: eck48gxq
Sweep URL: https://wandb.ai/jdc690nyu/Iv2Bothtest/sweeps/eck48gxq


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionResNetV2
import wandb
from wandb.keras import WandbCallback
import statistics


# Step 1: Define the InceptionV2 architecture
def WBcreate_inceptionv2_model(input_shape, num_classes, learning_rate, dropout):
    base_model = tf.keras.applications.InceptionResNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet')
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Step 2: Train the model
def WBtrain(config=None):
    input_shape = (224, 224, 3)
    num_classes = len(np.unique(y))

    with wandb.init(config=config):
        config = wandb.config
        lr = config.learning_rate
        batch_size = config.batch_size
        dropout = config.dropout

        inceptionv2_model = WBcreate_inceptionv2_model(input_shape, num_classes, learning_rate=lr, dropout=dropout)

        history = inceptionv2_model.fit(train_generator,
                                        epochs=10,
                                        validation_data=test_generator,
                                        batch_size=batch_size,
                                        callbacks=[WandbCallback()])
        
        # Save loss value
        wandb.log({"val_loss": statistics.mean(history.history['val_loss'])})
        wandb.log({"val_accuracy": statistics.mean(history.history['val_accuracy'])})


In [ ]:
wandb.agent(sweep_id, WBtrain, count=5)

wandb: Agent Starting Run: f4t7kw0s with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0
wandb: 	learning_rate: 0.00025
wandb: Currently logged in as: jdc690 (jdc690nyu). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 0.4328 - accuracy: 0.8701

wandb: Adding directory to artifact (/content/wandb/run-20230510_160340-f4t7kw0s/files/model-best)... Done. 3.8s


150/150 [==============================] - 327s 1s/step - loss: 0.4328 - accuracy: 0.8701 - val_loss: 0.3060 - val_accuracy: 0.9137
Epoch 2/10
150/150 [==============================] - 108s 718ms/step - loss: 0.1099 - accuracy: 0.9651 - val_loss: 0.4286 - val_accuracy: 0.9208
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: 0.0893 - accuracy: 0.9730

...

150/150 [==============================] - 220s 1s/step - loss: 0.0705 - accuracy: 0.9768 - val_loss: 0.1714 - val_accuracy: 0.9504


accuracy,▁▅▇▇▇█████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▂▂▂▁▁▁▁▁
val_accuracy,▁▆▄█▇▄█▇██▆
val_loss,█▂▃▁▂▃▁▂▁▁▃
accuracy,0.97676
best_epoch,9
best_val_loss,0.17143
epoch,9
loss,0.07049
val_accuracy,0.90817


wandb: Agent Starting Run: ss9tfpug with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.0005


Epoch 1/10


wandb: Ctrl + C detected. Stopping sweep.


## Single Convolution

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='SingleCNNBothtest')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: c99oevf1
Sweep URL: https://wandb.ai/jdc690nyu/SingleCNNBothtest/sweeps/c99oevf1


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbCallback
import statistics

# Step 1: Define the model architecture
def WBcreate_cnn_model(input_shape, num_classes, learning_rate, dropout):
    model = tf.keras.models.Sequential()

    # Convolutional layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the feature maps
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Step 2: Train the model
def WBtrain(config=None):
    input_shape = (224, 224, 3)
    num_classes = len(np.unique(y))

    with wandb.init(config=config):
        config = wandb.config
        lr = config.learning_rate
        batch_size = config.batch_size
        dropout = config.dropout

        cnn_model = WBcreate_cnn_model(input_shape, num_classes, learning_rate=lr, dropout=dropout)

        history = cnn_model.fit(train_generator,
                                epochs=10,
                                validation_data=test_generator,
                                batch_size=batch_size,
                                callbacks=[WandbCallback()])
        
        # Save loss value
        wandb.log({"val_loss": statistics.mean(history.history['val_loss'])})
        wandb.log({"val_accuracy": statistics.mean(history.history['val_accuracy'])})


In [ ]:
wandb.agent(sweep_id, WBtrain, count=5)

wandb: Agent Starting Run: w2sa5zr4 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.0005
wandb: Currently logged in as: jdc690 (jdc690nyu). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 3.6016 - accuracy: 0.0486

wandb: Adding directory to artifact (/content/wandb/run-20230510_162732-w2sa5zr4/files/model-best)... Done. 3.4s


150/150 [==============================] - 124s 753ms/step - loss: 3.6016 - accuracy: 0.0486 - val_loss: 2.9954 - val_accuracy: 0.0508
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 2.9954 - accuracy: 0.0518

...

wandb: Adding directory to artifact (/content/wandb/run-20230510_174125-6g6okmkb/files/model-best)... Done. 3.5s


150/150 [==============================] - 107s 714ms/step - loss: 2.9957 - accuracy: 0.0520 - val_loss: 2.9956 - val_accuracy: 0.0521
Epoch 10/10
150/150 [==============================] - ETA: 0s - loss: 2.9957 - accuracy: 0.0520

wandb: Adding directory to artifact (/content/wandb/run-20230510_174125-6g6okmkb/files/model-best)... Done. 3.5s


150/150 [==============================] - 110s 737ms/step - loss: 2.9957 - accuracy: 0.0520 - val_loss: 2.9956 - val_accuracy: 0.0521


accuracy,▆▁▃███████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁█████████▇
val_loss,█▇▆▅▅▄▃▂▂▁▄
accuracy,0.05198
best_epoch,9
best_val_loss,2.99564
epoch,9
loss,2.99567
val_accuracy,0.05196
